In [1]:
#Python Implementation of the Hartree Fock Method
#Procedures listed in the code follow as described in Modern Quantum Chemistry: 
#Introduction to Advanced Electronic Structure Theory, By Attila Szabo and Neil S. Ostlund
import math
import sys
import numpy as np
from molecule import atom
from molecule import vector
from molecule import gaussian
from molecule import molecule
from notebookImporter import importNotebook

#import integrals notebook for the hartree method
integrals = importNotebook("hartree_integrals")
scf = importNotebook("hartree_scf")

#define SCF convergence critera, and max number of iteration cycles
SCF_CONVERGENCE = pow(10, -100)
MAX_ITERATIONS = 100

#Step 1
#Specify Molecules, Nuclear Coordinates, and Charge of the nucli Number of Electrons,

#generate an h2 atom with a distance of 1.4 AU to compare with Szabo pg. 160
R = 1.4
system = molecule()
system.addAtom(atom(vector(1,1,1), 2, 2))
system.addAtom(atom(vector(1,1,1+R), 1,1))

#add a basis set
system.addBasis("6-21G")

system.display()

Molecule
Basis Set: 6-21G
Total Number of Electrons: 3
Atoms: 
     Atomic Number: 2, Electrons: 2, Coordinate: X: 1 Y: 1 Z: 1
     Atomic Number: 1, Electrons: 1, Coordinate: X: 1 Y: 1 Z: 2.4


In [2]:
#Step 2
#Calculate Integrals
#Overlap, KE, Nuclear Attraaction, and electron repulsion
S = integrals.overlap(system)
print("Overlap Matrix: ")
print(np.matrix(S))
print()

T = integrals.kineticEnergy(system)
print("Electron Kinetic Energy Matrix: ")
print(np.matrix(T))
print()

V = integrals.nuclearAttraction(system)
for index, atom in enumerate(V):
    print("Nucli " + str(index) + "-Electron Attraction Matrix: ")
    print(np.matrix(atom))
print()

electronRepulsion = integrals.electronElectronRepulsion(system)
print("Electron Repulsion Tensor: ")
print(np.array(electronRepulsion))
print()

#Form the electronic hamiltonian
H = np.matrix(T)

#add in all of the nuclear attractions matricies to the hamiltonian
for atom in V:
    H += np.matrix(atom)
        
print("Electronic Hamiltonian :")
print(H)
print()

Overlap Matrix: 
[[1.         0.59521595 0.24078493 0.27903177]
 [0.59521595 1.         0.5135924  0.70990153]
 [0.24078493 0.5135924  1.         0.64589812]
 [0.27903177 0.70990153 0.64589812 1.        ]]

Electron Kinetic Energy Matrix: 
[[3.9161238  0.5789494  0.06527748 0.10995836]
 [0.5789494  0.5744895  0.26673973 0.22117837]
 [0.06527748 0.26673973 1.54940495 0.29315085]
 [0.10995836 0.22117837 0.29315085 0.27478737]]

Nucli 0-Electron Attraction Matrix: 
[[-5.48968847 -2.23529223 -0.83414691 -0.98929468]
 [-2.23529223 -1.97512902 -0.91943322 -1.16038696]
 [-0.83414691 -0.91943322 -1.41570339 -0.88168287]
 [-0.98929468 -1.16038696 -0.88168287 -1.09892616]]
Nucli 1-Electron Attraction Matrix: 
[[-0.71424282 -0.4242527  -0.24436929 -0.2152917 ]
 [-0.4242527  -0.65490881 -0.63112353 -0.51455539]
 [-0.24436929 -0.63112353 -1.72091788 -0.78266155]
 [-0.2152917  -0.51455539 -0.78266155 -0.68300331]]

Electron Repulsion Tensor: 
[[[[1.80325459 0.87650682 0.33671087 0.39544301]
   [0.87

In [3]:
#Prepare for the SCF procedure

#get size of the basis set 
size = len(S)

#compute the Transformation Matrix
X = scf.X(S, size)

#get guess Fock matrix, assume 2-electron term is equal to 0
F = H

In [4]:
# SCF Procedure 

#init list to store the energy from each iteration
#as well as a boolean to signify whether the loop has converged
E = []
converged = False

while( not converged ):
  
    #diagnolze the Fock matrix and convert it to MO basis 
    F = X.transpose() * F * X 
    
   #diagnolize the Fock Matrix to obtain the MOs and the their respective energies
    MOEnergy, MO = np.linalg.eigh(F)
    
    #Transform the MO basis MOs to an AO basis
    C = X * MO
    
    #compute the electron density, the two electron term, and then use G to compute the new Fock matrix
    P = scf.densityMatrix(C, system.N, size)
    G = scf.G(electronRepulsion, P, size)
    F = H + G
    
    #compute the new expectation energy 
    E.append(scf.expectationEnergy(H, F, P))
    
    #check if at least two SCF iterations have occured
    #if more than two have occured, then check if the difference betweeen this E, 
    #and the previous E is less then the covergence value, if yes, end the SCF loop
    #if energy has not converged, check whether the max number of iterations have occured so far
    sizeE = len(E)
    if(len(E) > 2):
        if(abs(E[sizeE-2] - E[sizeE-1]) < SCF_CONVERGENCE):
            converged = True
        elif(sizeE > MAX_ITERATIONS):
            print("SCF Failed to Converge")
            break
    
    #compute total energy of the system including nuclear-nuclear repulsion
    totalE = E[sizeE-1] + scf.nuclearRepulsion(system)
    
    #display information about current SCF iteration to the user
    print("SCF Iteration #" + str(sizeE) + ", Electronic Energy: " + str(E[sizeE-1]) + " Hartrees, Total Energy: " + str(totalE) + " Hartrees")

[-2.70692115 -1.32641131 -0.48646321  0.1462177 ]


SCF Iteration #1, Electronic Energy: -4.211116231043732 Hartrees, Total Energy: -2.7825448024723034 Hartrees
[-1.55252674 -0.2354201   0.56147615  1.80562846]


SCF Iteration #2, Electronic Energy: -4.314511994098726 Hartrees, Total Energy: -2.8859405655272976 Hartrees
[-1.64771568 -0.23714703  0.55822297  1.71396999]


SCF Iteration #3, Electronic Energy: -4.314992868163754 Hartrees, Total Energy: -2.8864214395923256 Hartrees
[-1.6532159  -0.23761129  0.55824422  1.70675293]


SCF Iteration #4, Electronic Energy: -4.31500040632616 Hartrees, Total Energy: -2.886428977754731 Hartrees
[-1.65366117 -0.23765045  0.55829034  1.70609021]


SCF Iteration #5, Electronic Energy: -4.3150006663960125 Hartrees, Total Energy: -2.8864292378245837 Hartrees
[-1.65371469 -0.23765225  0.55830464  1.70600008]


SCF Iteration #6, Electronic Energy: -4.3150006764480855 Hartrees, Total Energy: -2.8864292478766567 Hartrees
[-1.65372336 -0.23765182  0.558308